## Import balíčků

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

## Načtení souborů

In [12]:
df_file_all = pd.read_csv('data/dc_file_modified2.csv')

df_file = df_file_all.tail(10000)

/tmp/ipykernel_8207/4073509301.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_file_all = pd.read_csv('data/dc_file_modified2.csv')


## Promazání zbytečných příznaků

příznaky jsme po jednom prošli, většinou měli příliš mnoho hodnot, byly duplicitní nebo nebyly podstatné pro naši analýzu.

In [13]:
df_file = df_file.drop(columns=['Unnamed: 0', 'id', 'filename', 'extra', 'datetime', 'file_path', 'name', 'typef', 'dir_type', 'file_type', 'is_allocated', 'is_allocated0', 'sha_256', 'epochtime', 'hour', 'minute'])

## Přetypování příznaků

In [14]:
bools = [
         'M', 'A', 'C', 'B',
         'file_stat', 'NTFS_file_stat', 'file_entry_shell_item', 'NTFS_USN_change',
         'filef', 'directory', 'link',
         'dir_appdata', 'dir_win', 'dir_user', 'dir_other',
         'file_executable', 'file_graphic', 'file_documents', 'file_ps', 'file_other',
         'mft', 'lnk_shell_items', 'olecf_olecf_automatic_destinations/lnk/shell_items', 'winreg_bagmru/shell_items', 'usnjrnl',
         'is_allocated1'
         ]

df_file['timestamp'] = df_file['timestamp'].astype('datetime64[ns]')

df_file[bools] = df_file[bools].astype('boolean')

# procento aktivit v mesici, kdy doslo k inidentu
# 100 * (df_file['timestamp'] >= '2020-09-01').sum() / df_file.shape[0]
dict(df_file.dtypes)

{'inode': dtype('int64'),
 'M': BooleanDtype,
 'A': BooleanDtype,
 'C': BooleanDtype,
 'B': BooleanDtype,
 'file_stat': BooleanDtype,
 'NTFS_file_stat': BooleanDtype,
 'file_entry_shell_item': BooleanDtype,
 'NTFS_USN_change': BooleanDtype,
 'filef': BooleanDtype,
 'directory': BooleanDtype,
 'link': BooleanDtype,
 'dir_appdata': BooleanDtype,
 'dir_win': BooleanDtype,
 'dir_user': BooleanDtype,
 'dir_other': BooleanDtype,
 'file_executable': BooleanDtype,
 'file_graphic': BooleanDtype,
 'file_documents': BooleanDtype,
 'file_ps': BooleanDtype,
 'file_other': BooleanDtype,
 'mft': BooleanDtype,
 'lnk_shell_items': BooleanDtype,
 'olecf_olecf_automatic_destinations/lnk/shell_items': BooleanDtype,
 'winreg_bagmru/shell_items': BooleanDtype,
 'usnjrnl': BooleanDtype,
 'is_allocated1': BooleanDtype,
 'file_size': dtype('int64'),
 'timestamp': dtype('<M8[ns]')}

### Příznak file_size

změníme file_size podle toho, do jakého intervalu spadá
```
0: 0
1: 1 - 1_000
2: 1_001 - 10_000
3: 10_001 - 100_000
4: 100_001 - 1_000_000
5: 1_000_001+
```

In [15]:
df_file['file_size+1_log'] = np.log(df_file['file_size'] + 1)

size_stamps = [-float('inf'), 0, 1_000, 10_000, 100_000, 1_000_000, float('inf')]
size_labels = ['0', '1-1_000', '1_001-10_000', '10_001-100_000', '100_001-1_000_000', '1_000_000+']
# make data ordinal based on size

df_file['file_size'] = pd.cut(df_file['file_size'], size_stamps, labels=False)
df_file['file_size'].value_counts()

0    9541
2     163
3      97
1      89
5      65
4      45
Name: file_size, dtype: int64

In [16]:
df_file.sample(10)

,inode,M,A,C,B,file_stat,NTFS_file_stat,file_entry_shell_item,NTFS_USN_change,filef,...,file_other,mft,lnk_shell_items,olecf_olecf_automatic_destinations/lnk/shell_items,winreg_bagmru/shell_items,usnjrnl,is_allocated1,file_size,timestamp,file_size+1_log
843859,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-19 04:52:45,0.0
834489,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-18 22:25:35,0.0
841859,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-19 03:30:42,0.0
836388,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-18 22:37:30,0.0
838062,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-18 23:07:09,0.0
841940,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-19 03:30:42,0.0
839963,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-19 02:08:28,0.0
839604,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-19 01:43:25,0.0
838118,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-18 23:07:21,0.0
842970,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-19 04:07:38,0.0


In [19]:
#? tohle by bylo nice, ale je to moc pomale
def find_minute_frequency(row):
    return df_file[(df_file['timestamp'] >= row['timestamp'] - pd.Timedelta(minutes=1)) \
                   & (df_file['timestamp'] < row['timestamp'] + pd.Timedelta(minutes=1))].shape[0]

minute_activity = df_file.apply(find_minute_frequency, axis=1)

In [20]:
# floored_to_min = df_file['timestamp'].dt.floor('min')
# counts = floored_to_min.value_counts().to_dict()

# minute_activity = df_file['timestamp'].apply(lambda x: counts[x.floor('min')])
# minute_activity.to_csv('data/minute_activity.csv')
# # minute_activity = pd.read_csv('data/minute_activity.csv', index_col=0)['timestamp']

In [21]:
df_file['minute_activity'] = minute_activity

In [22]:
inode_counts = df_file['inode'].value_counts().to_dict()

inode_activity = df_file['inode'].apply(lambda x: inode_counts[x])

In [23]:
df_file['inode_activity'] = inode_activity
df_file.drop(columns=['inode', 'timestamp'], inplace=True)
df_file['epochtime'] = df_file_all['epochtime'].astype('int64')

In [24]:
df_file.to_csv('data/dc_file_preprocessed.csv', index=False)

## Standardizace

In [25]:
standard_scaler = StandardScaler()

In [26]:
df_file_standardized = pd.DataFrame(standard_scaler.fit_transform(df_file), columns=df_file.columns)

In [27]:
df_file_standardized.to_csv('data/dc_file_preprocessed_standardized.csv', index=False)